In [1]:
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever


In [2]:
import os
YOUR_PATH=os.getcwd()
document_store = FAISSDocumentStore.load(f"{YOUR_PATH}/faiss2_document_store")

In [3]:
# Load FAISS document store
#document_store = FAISSDocumentStore.load(index_path="PSSCOC1_index.faiss",config_path="PSSCOC1_config.json")
#document_store = FAISSDocumentStore(sql_url="sqlite:///my_doc_store.db",faiss_config_path="PSSCOC1_config.json")
#documents = document_store.get_all_documents(return_embedding=True)
#print(documents)

In [4]:
from haystack.nodes import EmbeddingRetriever

# Initialize EmbeddingRetriever Retriever to encode documents, encode question and query documents
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)

C:\Users\Admin\anaconda3\envs\QuestionAnswer\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
QUESTIONS = [
    "What must happen for the Employer to terminate the contract in PSSCOC?",
    "What happens when a Contractor is terminated for default in PSSCOC?",
    "What are the liquidated damages after termination in PSSCOC?",
    "Is the contractor responsible for damages caused by the subcontractor in PSSCOC?",
    "Write me a contract provision on a Cafe within an underground rail system"
]

In [6]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

In [7]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What must happen for the Employer to terminate the contract in PSSCOC?
Answers:
[   {   'answer': 'It depends on the contract. If the Contractor has failed to '
                  'complete the work in accordance with the terms of the '
                  'contract, then the Employer has the right to terminate the '
                  'contract at any time.'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What happens when a Contractor is terminated for default in PSSCOC?
Answers:
[   {   'answer': 'It depends on the contract. If the contract says that the '
                  'Contractor is responsible for all of the work done by the '
                  'subcontractor, then the employer will be responsible for '
                  'the work performed by that subcontractor. The Contractor '
                  'will not be held responsible for any work that is performed '
                  'by a subcontractor that is not part of the Contract.'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What are the liquidated damages after termination in PSSCOC?
Answers:
[   {   'answer': 'The amount of liquidated damages is determined by the terms '
                  'of the contract. For example, if the contract says that the '
                  'Contractor must complete the work by the time the time for '
                  'Completion expires, then the contract is void. The '
                  'Contractor is responsible for the cost of completing the '
                  'work. If the contractor does not complete the works by the '
                  'end of the time period specified in the contract, the '
                  'contract remains void.'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Is the contractor responsible for damages caused by the subcontractor in PSSCOC?
Answers:
[   {   'answer': 'Yes, the contractor is responsible for any damages caused '
                  'by the subcontractor in PSSCOC.'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Write me a contract provision on a Cafe within an underground rail system
Answers:
[   {   'answer': "I'm not a lawyer, but I'm pretty sure this is what you're "
                  'looking for.'}]
